# Climate Risk Visualization for Ames, Iowa

This script analyzes climate risks across neighborhoods in Ames, Iowa, and visualizes them using a heatmap in Folium.

## Steps:
1. **Load Data**: Read the dataset (`train.csv`) using Pandas.
2. **Risk Scoring**: Assign risk levels based on `LandContour`, `LandSlope`, `RoofMatl`, and `Exterior1st`.
3. **Aggregate by Neighborhood**: Compute average flood risk, fire risk, and slope steepness.
4. **Geographical Mapping**: Assign latitude and longitude to each neighborhood.
5. **Normalize Total Risk**: Scale the composite risk score between 0 and 1.
6. **Generate Heatmap**: Use Folium's `HeatMap` to display risk levels with a gradient from blue (low) to red (high).
7. **Add Neighborhood Labels**: Display risk scores on the map with popups.
8. **Legend Addition**: Create a color-coded risk legend for better interpretation.
9. **Save and Display Map**: Output the final interactive map (`climate_risk_map_with_legend.html`).

### Output:
The resulting map visually represents climate risks across different neighborhoods, helping in better urban planning and risk assessment.


In [8]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from branca.element import Template, MacroElement

# Load dataset
df = pd.read_csv('train.csv')

# Define risk scoring mappings for different parameters
risk_mapping = {
    'LandContour': {'Low': 3, 'HLS': 2, 'Bnk': 1, 'Lvl': 0},  # Higher values indicate more flood-prone areas
    'LandSlope': {'Sev': 2, 'Mod': 1, 'Gtl': 0},  # Steeper slopes are riskier
    'RoofMatl': {'WdShngl': 3, 'WdShake': 3, 'Metal': 1, 'CompShg': 2, 'Tar&Grv': 1, 'Roll': 0},  # Fire risk based on roofing material
    'Exterior1st': {'Wd Sdng': 2, 'WdShing': 2, 'VinylSd': 1, 'Metal': 0, 'CemntBd': 0}  # Fire risk based on exterior material
}

# Calculate flood risk and fire risk scores based on predefined mappings
df['Flood_Risk'] = df['LandContour'].map(risk_mapping['LandContour']) + \
                   df['LandSlope'].map(risk_mapping['LandSlope'])

df['Fire_Risk'] = df['RoofMatl'].map(risk_mapping['RoofMatl']) + \
                  df['Exterior1st'].map(risk_mapping['Exterior1st'])

# Aggregate risk values by neighborhood
climate_risk = df.groupby('Neighborhood').agg({
    'Flood_Risk': 'mean',  # Average flood risk per neighborhood
    'Fire_Risk': 'mean',  # Average fire risk per neighborhood
    'LandSlope': lambda x: (x == 'Sev').mean()  # Percentage of severely sloped areas as a proxy for steepness
}).reset_index()

# Define latitude and longitude for each neighborhood
neighborhood_coords = {
    'Blmngtn': (42.079, -93.620), 'Blueste': (42.067, -93.639),
    'BrDale': (42.052, -93.619), 'BrkSide': (42.034, -93.615),
    'ClearCr': (42.061, -93.629), 'CollgCr': (42.019, -93.651),
    'Crawfor': (42.026, -93.631), 'Edwards': (42.023, -93.685),
    'Gilbert': (42.105, -93.649), 'IDOTRR': (42.018, -93.612),
    'MeadowV': (42.034, -93.655), 'Mitchel': (42.067, -93.640),
    'NAmes': (42.042, -93.615), 'NoRidge': (42.073, -93.660),
    'NPkVill': (42.050, -93.643), 'NridgHt': (42.050, -93.656),
    'NWAmes': (42.056, -93.675), 'OldTown': (42.027, -93.613),
    'SWISU': (42.018, -93.651), 'Sawyer': (42.023, -93.656),
    'SawyerW': (42.024, -93.665), 'Somerst': (42.065, -93.639),
    'StoneBr': (42.076, -93.635), 'Timber': (42.030, -93.669),
    'Veenker': (42.073, -93.660)
}

# Assign latitude and longitude to each neighborhood
climate_risk['Latitude'] = climate_risk['Neighborhood'].map(lambda x: neighborhood_coords.get(x, (42.0308, -93.6313))[0])
climate_risk['Longitude'] = climate_risk['Neighborhood'].map(lambda x: neighborhood_coords.get(x, (42.0308, -93.6313))[1])

# Create a normalized total risk score (scaled between 0 and 1)
climate_risk['Total_Risk'] = (climate_risk[['Flood_Risk', 'Fire_Risk', 'LandSlope']]
                              .mean(axis=1)
                              .transform(lambda x: (x - x.min()) / (x.max() - x.min())))

# Initialize a Folium map centered at Ames, Iowa
ames_map = folium.Map(location=[42.0308, -93.6313], zoom_start=13)

# Add heatmap for climate risk visualization
HeatMap(
    data=climate_risk[['Latitude', 'Longitude', 'Total_Risk']].values.tolist(),
    radius=20,  # Spread of each data point
    blur=15,  # Softness of the heatmap
    min_opacity=0.4,  # Transparency of low-risk areas
    gradient={'0.4': 'blue', '0.6': 'lime', '0.8': 'orange', '1.0': 'red'}  # Color mapping for risk levels
).add_to(ames_map)

# Add markers with popups displaying risk details for each neighborhood
for idx, row in climate_risk.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"""
        <b>{row['Neighborhood']}</b><br>
        Total Risk: {row['Total_Risk']:.2f}<br>
        Flood Risk: {row['Flood_Risk']:.2f}<br>
        Fire Risk: {row['Fire_Risk']:.2f}<br>
        Steep Slope: {row['LandSlope']:.0%}
        """,
        icon=folium.DivIcon(html=f"<div style='font-size: 8pt'>{row['Neighborhood']}</div>")
    ).add_to(ames_map)

# Define a legend for better interpretation
template = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 200px;
    height: 200px;
    z-index:9999;
    font-size:12px;
    background-color:white;
    padding:15px;
    border-radius:5px;
    box-shadow: 0 0 5px rgba(0,0,0,0.2);
    ">
    <p style='margin:0;font-weight:bold'>Climate Risk Legend</p>
    <div style="background: linear-gradient(to right,
        blue 0%,
        lime 33%,
        orange 66%,
        red 100%);
        height: 25px;
        margin: 10px 0">
    </div>
    <div style="display: flex; justify-content: space-between">
        <span>Low</span>
        <span>Medium</span>
        <span>High</span>
    </div>
    <p style='margin:5px 0'>Color Key:</p>
    <div style="display: grid; grid-template-columns: 20px auto; gap: 2px">
        <div style="background: blue"></div><div>Low</div>
        <div style="background: lime"></div><div>Moderate</div>
        <div style="background: orange"></div><div>High</div>
        <div style="background: red"></div><div>Very High</div>
    </div>
</div>
{% endmacro %}
"""

# Add the legend to the map
macro = MacroElement()
macro._template = Template(template)
ames_map.get_root().add_child(macro)

# Save and display the map
ames_map.save('climate_risk_map_with_legend.html')
ames_map
